In [ ]:
from datetime import datetime, timedelta
mindate = datetime.strptime('2021-01-01','%Y-%m-%d')
maxdate = datetime.strptime('2021-12-31','%Y-%m-%d')
daterange = [ (mindate + timedelta(hours=x)).strftime('%Y%m%d %H') for x in range(0,(maxdate-mindate).days)]

print(daterange[:10])



In [ ]:
import pandas as pd
dt_index = pd.date_range(start='20210101', end='20211231')
dt_list = dt_index.strftime('%Y-%m-%d').tolist()
print(dt_list[:10])

In [ ]:
# tier1 데이터 sql
from pyspark.sql import SparkSession
spark = SparkSession.builder \
            .appName('test-session') \
            .master('local[*]') \
            .getOrCreate()
print(spark)
data_path = '/home/data/data-warehouse/tier1/event/2020-12-01/00'
attr = [
    '*',
]
df = spark.read.format('parquet') \
    .option('header',True) \
    .option('compression', 'gzip') \
    .load(data_path) \
    .selectExpr(attr)
df.show(3)

In [ ]:
# raw 데이터 sql
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = SparkSession.builder \
            .appName('test-session') \
            .master('local[*]') \
            .getOrCreate()
print(spark)
data_path = '/home/data/dmc-integrated-analytics/rawdata/thirdparty-branch/2020-12/01/00/*'
# data_path = '/home/data/dmc-integrated-analytics/rawdata/thirdparty-adbrix/2020-12/01/00/*'
# data_path = '/home/data/dmc-integrated-analytics/rawdata/thirdparty-singular/2021-01/01/00/*'
pre_branch_attr = [
    '"BRANCH"                                        AS tracking',
    'trackingObject.organization_name[0]             AS trackingId',
    'CASE WHEN trackingObject.name[0] = "SEARCH"   THEN "TPD007" \
            WHEN trackingObject.name[0] = "VIEW_ITEM"  THEN "TPD002" \
            WHEN trackingObject.name[0] = "ADD_TO_CART" THEN "TPD003" \
            WHEN trackingObject.name[0] = "PURCHASE" THEN "TPD004"\
            WHEN trackingObject.name[0] = "INSTALL"    THEN "TPD008"\
            WHEN trackingObject.name[0] = "OPEN" THEN "TPD009"  ELSE "TPD999" END     AS trackingEventCode',
    'CASE WHEN trackingObject.ud_os[0] = "ANDROID" THEN trackingObject.ud_aaid[0] \
            WHEN trackingObject.ud_os[0] = "IOS" THEN trackingObject.ud_idfa[0] END                     AS cid',
    'CASE WHEN trackingObject.ud_os[0] = "ANDROID" THEN "ATC001" \
            WHEN trackingObject.ud_os[0] = "IOS" THEN "ATC002" ELSE "ATC999" END      AS osTypeCode',
    'logTimeStamp                  ',
    'trackingObject.event_timestamp[0]               AS eventTimeStamp',
    'trackingObject.latd_campaign[0]                 AS campaign',
    'trackingObject.ci0_product_name[0]              AS contentId',
    '""                                              AS contentName',
    'trackingObject.ci0_price[0]                     AS value',
    'trackingObject.ci0_quantity[0]                  AS quantity',
    'trackingObject.ci0_price[0] * trackingObject.ci0_quantity[0]     AS amount',
    'trackingObject.ci0_currency[0]                  AS currency',
    '""                                              AS activityParam',
    'trackingObject.attributed[0]                    AS attributed',
    'trackingObject.latd_advertising_partner_name[0] AS latdAdvertisingPartnerName',
]
pre_adbrix_attr = [  
    '"ADBRIX"                                      AS tracking',
    'trackingObject.package_name[0]                AS trackingId',      
    'CASE WHEN trackingObject.activity[0] = "search"  THEN "TPD001" \
            WHEN trackingObject.activity[0] = "product_view"  THEN "TPD002" \
            WHEN trackingObject.activity[0] = "add_to_cart" THEN "TPD003" \
            WHEN trackingObject.activity[0] = "purchase" THEN "TPD004" \
            WHEN trackingObject.activity[0] = "sign_up" THEN "TPD012" \
            WHEN trackingObject.activity[0] = "refund" THEN "TPD015" \
            WHEN trackingObject.activity[0] = "add_to_wishlist" THEN "TPD014" \
            WHEN trackingObject.activity[0] = "login" THEN "TPD013" ELSE "TPD999" END  AS trackingEventCode',
    'CASE WHEN trackingObject.platform[0] = "android" THEN trackingObject.gaid[0] ELSE trackingObject.ifa[0] END AS cid',
    'CASE WHEN trackingObject.platform[0] = "android" THEN "ATC001" \
            WHEN trackingObject.platform[0] = "ios" THEN "ATC002" ELSE "ATC999" END AS osTypeCode', 
    'CAST(logTimeStamp AS timestamp) AS logTimeStamp',
    'CAST(trackingObject.event_time[0] * 1000 AS timestamp)    AS eventTimeStamp',
    '""                                            as campaign',
    'trackingObject.product_id[0]                  AS contentId',
    'trackingObject.product_name[0]                AS contentName',
    'cast(trackingObject.price[0] as integer)                       AS value',
    'cast(trackingObject.quantity[0] as integer)                    AS quantity',
    'cast(trackingObject.sales[0] as integer)                       AS amount',
    'trackingObject.currency[0]                    AS currency',
    'trackingObject.activity_param[0]              AS activityParam',
    '""                                            AS attributed',
    '""                                            AS latdAdvertisingPartnerName',      
]
pre_singular_attr = [
    '"SINGULAR"                                               AS tracking',
    'trackingObject.app_name[0]                               AS trackingid',
    'CASE WHEN trackingObject.evtname[0] = "ViewItem"  THEN "TPD002"\
            WHEN trackingObject.evtname[0] = "AddToCart" THEN "TPD003"\
            WHEN trackingObject.evtname[0] = "Purchase"  THEN "TPD004"\
            WHEN trackingObject.evtname[0] = "FirstPurchase"  THEN "TPD010"\
            WHEN trackingObject.evtname[0] = "__START__" THEN "TPD008"\
            WHEN trackingObject.evtname[0] = "__SESSION__"  THEN "TPD009"\
            WHEN trackingObject.evtname[0] = "__iap__"   THEN "TPD011"   ELSE "TPD999" END AS trackingEventCode',
    'CASE WHEN trackingObject.platform[0] = "Android" THEN trackingObject.aifa[0] ELSE trackingObject.idfa[0] END AS cid',
    'CASE WHEN trackingObject.platform[0] = "Android" THEN "ATC001"\
            WHEN trackingObject.platform[0] = "iOS" THEN "ATC002"  ELSE "ATC999" END  AS osTypeCode',
    'logTimeStamp',
    'CAST(trackingObject.utc[0] * 1000 AS String)             AS eventTimeStamp',
    'trackingObject.cid[0]                                    AS campaign',
    'CASE WHEN trackingObject.evtname[0] = "__iap__" THEN GET_JSON_OBJECT(trackingObject.evtattr[0], "$[0].id") ELSE GET_JSON_OBJECT(trackingObject.evtattr[0], "$[0].prd_code") END AS contentId',
    'GET_JSON_OBJECT(trackingObject.evtattr[0], "$[0].product")  AS contentName',
    'CASE WHEN trackingObject.evtname[0] = "__iap__" THEN GET_JSON_OBJECT(trackingObject.evtattr[0], "$[0].price") ELSE "0" END  AS value',
    'CASE WHEN trackingObject.evtname[0] = "__iap__" THEN GET_JSON_OBJECT(trackingObject.evtattr[0], "$[0].quantity") ELSE "0" END AS quantity',
    'trackingObject.amount[0]                                 as amount',
    'trackingObject.currency[0]                               as currency',
    'trackingObject.evtattr[0]                                AS activityParam',
    'trackingObject.is_re_eng[0]                              AS attributed',
    '""                                                       AS latdAdvertisingPartnerName',
]

# 스키마 지정해서 불러들여야하는데... 에러남
union_schema = StructType([
    StructField('tracking', StringType(), True),
    StructField('trackingId', StringType(), True),
    StructField('trackingEventCode', StringType(), True),
    StructField('cid', StringType(), True),
    StructField('osTypeCode', StringType(), True),
    StructField('logTimeStamp', TimestampType(), True),
    StructField('eventTimeStamp', TimestampType(), True),
    StructField('campaign', StringType(), True),
    StructField('contentId', StringType(), True),
    StructField('contentName', StringType(), True),
    StructField('value', IntegerType(), True),
    StructField('quantity', IntegerType(), True),
    StructField('amount', IntegerType(), True),
    StructField('currency', StringType(), True),
    StructField('activityParam', StringType(), True),
    StructField('attributed', StringType(), True),
    StructField('latdAdvertisingPartnerName', StringType(), True),
])


df = spark.read\
    .format('json')\
    .option('header',True)\
    .load(data_path)\
    .selectExpr(pre_adbrix_attr)\

In [ ]:
df.printSchema()

In [ ]:
df.show(3,False)

In [ ]:
# write
write_path = '/home/data/temp/adbrix'
df.where('trackingId is not NULL')\
    .sort('eventTimeStamp')\
    .coalesce(20)\
    .write\
    .option('compression','gzip')\
    .option('parquet.enable.dictionary', 'true')\
    .option('parquet.block.size', f'{128*1024*1024}')\
    .option('parquet.page.size', f'{2*1024*1024}')\
    .option('parquet.dictionary.page.size', f'{8*1024*1024}')\
    .mode('overwrite')\
    .parquet(write_path)

In [ ]:
# read and print schema
df2 = spark.read\
    .format('parquet')\
    .option('header',True)\
    .load(write_path)    

df2.printSchema()

In [ ]:
# 파일폴더 변경..
import os
import shutil
from distutils.dir_util import copy_tree

In [ ]:
os.path.isdir('/home/data/data-warehouse/tier1/event/2021-01-01')

In [ ]:
import os, shutil
def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)        
        if os.path.isdir(d):            
            for file in os.listdir(s):
                # print(os.path.join(s, file),os.path.join(d, file))
                shutil.copy2(os.path.join(s, file),os.path.join(d, file))
        else:
            shutil.copytree(s, d, symlinks, ignore)

In [ ]:
rootdir = '/home/data/data-warehouse/tier1/event'
dst = '/home/data/data-warehouse/tier1_temp/event/'
for i,dirdate in enumerate(os.listdir(rootdir)) :
    # if i > 2 : break
    src = f'{rootdir}/{dirdate}/'
    print(src)
    copytree(src, dst)



In [ ]:
sql =''' select cast(avg(100000000000000) as bigint) as num '''
spark.sql(sql).show(5,False)

In [ ]:
data_path = '/home/data/dmc-integrated-analytics/record/advertiser/app/*'
spark.read.format('parquet').option('header',True).option('compression','gzip').load(data_path).show()

In [15]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
            .appName('test-session') \
            .master('local[*]') \
            .getOrCreate()
print(spark)

In [4]:
spark.sql('''
    select datediff(date('2021-10-01'),date('2021-05-01'))
''').show()

+--------------------------------+
|datediff(2021-10-01, 2021-05-01)|
+--------------------------------+
|                             153|
+--------------------------------+



In [7]:
spark.sql('''
    select round(0.021021,2)
''').show()

+------------------+
|round(0.021021, 2)|
+------------------+
|              0.02|
+------------------+



In [14]:
spark.sql('''
    select unix_timestamp(timestamp('2019-06-24 12:01:19.000')) - unix_timestamp(timestamp('2019-06-24 12:00:19.000'))
''').show()

+-----------------------------------------------------------------------------------------------------------------------------+
|(unix_timestamp(2019-06-24 12:01:19.000, yyyy-MM-dd HH:mm:ss) - unix_timestamp(2019-06-24 12:00:19.000, yyyy-MM-dd HH:mm:ss))|
+-----------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                           60|
+-----------------------------------------------------------------------------------------------------------------------------+



In [16]:
data_path = '/home/data/temp/15e3267b-7bc5-43d5-b723-6bc968a7e9db.parquet'
spark.read.format('parquet').option('header',True).load(data_path).show()

+-----------+----------------+----------------+---------------------+---------------+
|common_code|common_code_name|common_code_item|common_code_item_name|    description|
+-----------+----------------+----------------+---------------------+---------------+
|     TTC001|  USER_TYPE_CODE|          UTC001|               SYSTEM| 시스템 사용자 |
|     TTC001|  USER_TYPE_CODE|          UTC002|           ADVERTISER|         광고주|
|     TTC001|  USER_TYPE_CODE|          UTC003|            PUBLISHER|       퍼블리셔|
|     TTC002| EVENT_TYPE_CODE|          TPD001|             pageview|     페이지 뷰 |
|     TTC002| EVENT_TYPE_CODE|          TPD002|          viewcontent|      상품 보기|
|     TTC002| EVENT_TYPE_CODE|          TPD003|            addtocart|    카트에 담기|
|     TTC002| EVENT_TYPE_CODE|          TPD004|            purchased|       구매완료|
|     TTC002| EVENT_TYPE_CODE|          TPD005|         registration|       회원가입|
|     TTC002| EVENT_TYPE_CODE|          TPD006|              catalog|  상품 카탈로그|
|     TT

In [21]:
spark.sql('''
    with repeat_interval(index_name, interval_date) as (
        values('01 day repeat', 1)
        ,('02 day repeat', 2)
        ,('03 day repeat', 3)
        ,('04 day repeat', 4)
        ,('05 day repeat', 5)
        ,('06 day repeat', 6)
        ,('07 day repeat', 7)        
    )
    select * from repeat_interval
''').show()

+-------------+-------------+
|   index_name|interval_date|
+-------------+-------------+
|01 day repeat|            1|
|02 day repeat|            2|
|03 day repeat|            3|
|04 day repeat|            4|
|05 day repeat|            5|
|06 day repeat|            6|
|07 day repeat|            7|
+-------------+-------------+

